In [2]:
import random # library for random number generation
import pandas as pd
import requests
!pip install bs4
from bs4 import BeautifulSoup
!pip install geocoder
import geocoder
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [3]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'html.parser')

In [4]:
text = soup.find_all('table', attrs={'class':'wikitable sortable'})

In [5]:
texttemp2 = text[0].find_all('tr')

In [6]:
len(texttemp2)

181

In [7]:
result = []

for temp in texttemp2:
    
    temp2= temp.find_all('td')
    
    if (temp2!=[] and temp2[1].text[0:-1]!= 'Not assigned'):
        
        postalcode= temp2[0].text[0:-1]
        borough= temp2[1].text[0:-1]
        neighborhood= temp2[2].text[0:-1]
        
        if (neighborhood== 'Not assigned'):
            
            neighborhood= borough
            
        result.append((postalcode, borough, neighborhood)) 

In [8]:
df = pd.DataFrame(result, columns=['Postalcode', 'Borough', 'Neighborhood'])
df

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
df.shape

(103, 3)

In [93]:
dftemp= pd.DataFrame(columns=['Latitude','Longitude'])

for data, spec in zip (df['Borough'], df['Neighborhood']):
    
    splitted_string =spec.split(',')

    address= data +', '+ splitted_string [0] +', TO'
    geolocator = Nominatim(user_agent="to_explorer")
    location = geolocator.geocode(address)
    if (location== None):
        address= data +', TO'
        geolocator = Nominatim(user_agent="to_explorer")        
        location = geolocator.geocode(address)        
    latitude = location.latitude
    longitude = location.longitude
        
    dftemp= dftemp.append({'Latitude': latitude,
                           'Longitude': longitude}, ignore_index=True)
dftemp   

,Latitude,Longitude
0,43.761224,-79.323986
1,43.732658,-79.311189
2,43.654174,-79.380812
3,43.722079,-79.437507
4,43.654174,-79.380812
...,...,...
98,43.647381,-79.511333
99,43.660568,-79.383785
100,43.721789,-79.374027
101,43.651301,-79.495475


In [97]:
df = pd.concat([df, dftemp], axis=1)
df

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.761224,-79.323986
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654174,-79.380812
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722079,-79.437507
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.654174,-79.380812
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.647381,-79.511333
99,M4Y,Downtown Toronto,Church and Wellesley,43.660568,-79.383785
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.721789,-79.374027
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.651301,-79.495475


In [98]:
df.shape

(103, 5)

In this section i have clustered the postal code for highlighting hypothetical density of population

In [108]:
kclusters = 5

clustering = df

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dftemp)
clusters= kmeans.labels_
clusters

array([1, 1, 4, 3, 4, 2, 0, 1, 1, 4, 3, 2, 0, 1, 1, 4, 3, 2, 0, 4, 4, 3,
       0, 1, 4, 4, 0, 1, 3, 1, 4, 3, 0, 3, 3, 1, 4, 4, 0, 1, 3, 1, 4, 4,
       0, 1, 3, 1, 4, 3, 3, 0, 1, 3, 1, 1, 3, 3, 0, 1, 3, 1, 3, 3, 3, 0,
       1, 1, 4, 3, 3, 0, 1, 4, 4, 4, 2, 2, 0, 1, 4, 3, 0, 4, 4, 0, 4, 4,
       2, 2, 0, 4, 4, 2, 2, 0, 4, 4, 2, 4, 1, 2, 2], dtype=int32)

In [110]:
import numpy as np

TOlat= 43.6529
TOlong= -79.3849

map_clusters = folium.Map(location=[TOlat, TOlong], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Postalcode'], clusters):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters